# Star Trail Image Stacker

A rudimentary algorithm for merging multiple pictures of star trails into one, while "erasing" light clouds that have passed in front of regions of the picture for a short duration. 

Usually, multiple short-exposure photos taken of the night sky in clear weather can be directly merged into one using tools such as the "Lighten" layer blend mode in Adobe Photoshop, which creates a composite of only the brightest pixels at the same location in all the layers. However, if a small light-grey/white cloud moved across the frame, it obscures that entire region in the merged photo, because it may be lighter than the stars/sky behind it. 

To counteract this, the same Lighten blending can be applied only to selected pixels of a layer that contain a "star". Rather than training a neural network to identify stars in an image and create a binary mask of it with them selected, i found it simpler & quicker to just perform edge-detection in the picture. This selects the stars, which appear as lines/dots with clear contrast, but not clouds which have soft edges. The result is then used as a mask to lighten-blend the image in sequence with its previous one.

**Note** : This is an interactive notebook with widgets to adjust the parameters of the algorithm. These may not work/render if disabled in the Jupyter lab / another environment.

-------

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob, os

import ipywidgets as widgets
import IPython.display as disp

# list available backends
# 'widget' backend allows interactive plots within the notebook
# Matplotlib is convenient for zooming into the image and viewing pixel coordinates/values
%matplotlib -l      
%matplotlib widget


Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


#### 1. Select Images

- Specify the paths to input images as a list in `SOURCE_PATH`. If it is a serially numbered image sequence, wildcards such as `*`, `[]`, etc can be used with `glob` to generate the list automatically.
- `BASE_PATH` contains the path of a single image which will form the bottom-most layer of the image stack. All the other source images are blended together above this, and only their edge-regions will appear above it, while the rest of the background will be the same as that of this image.

The file upload widget itself can't be used to generate the list, it's just to browse the filesystem from here. Copy the address string of the files into the list.

In [2]:
SOURCE_PATH = glob.glob("./data/*.jpg")
BASE_PATH   = "./data/4J7A6511.jpg"

print("Current Working Directory :", os.getcwd())
widgets.FileUpload(accept='image/*', multiple=True)


Current Working Directory : /Users/gautamd/Home/github/startrail-merger


FileUpload(value={}, accept='image/*', description='Upload', multiple=True)

In [3]:
SOURCE_PATH.sort()

if not SOURCE_PATH:
    raise FileNotFoundError("Please set a valid image sequence path for SOURCE_PATH")
if not os.path.isfile(BASE_PATH):
    raise FileNotFoundError("Please set a valid image path for BASE_PATH")


_abs_source = list(map(os.path.abspath, SOURCE_PATH))
if os.path.abspath(BASE_PATH) in _abs_source:
    SOURCE_PATH.pop(_abs_source.index(os.path.abspath(BASE_PATH)))

In [4]:
def displayimg(img, title=None, close=None):
    """Use matplotlib to display an image"""
    
    fax = fig, ax = plt.subplots()
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if title :
        ax.set_title(title)
    fig.subplots_adjust(bottom=0, top=1, left=0.1, right=0.9)
    if isinstance(close, plt.Figure) :
        plt.close(close)
    return fax


#### 2. Slice

To reduce the time taken to re-process the entire image everytime any parameters are tweaked, you can choose to just process a sub-section of the image.
Specify

```SLICE = (x1, y1, x2, y2)``` 

to identify the top-left (x1, y1) and bottom-right (x2, y2) of the rectangular region to process.
- Use `SLICE = (0, 0, *baseImg.shape[:2])` to select the entire Image

In [5]:
baseImg = cv2.imread(BASE_PATH)

SLICE = (0, 0, *baseImg.shape[:2])
# SLICE = (2000, 500, 3500, 1500)

displayimg(baseImg[SLICE[1]:SLICE[2], SLICE[0]:SLICE[3]], "Base Image")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>,
 <AxesSubplot:title={'center':'Base Image'}>)

In [6]:
outline_kernel = np.array([[-1, -1, -1],
                           [-1, 8, -1],
                           [-1, -1, -1]])

dil_kernel = np.ones((3, 3), np.uint8)

#### 3. Blending

The following cell contains the code to blend the images. There are 4 adjustable parameters for creating the masks :
- `edge_tol` : A scalar that will be multiplied with the convolution kernel for outline selection.
- `blur_size` : Kernel size for smoothing the image after edge detection, reduces noise.
- `dil_iter` : Number of dilation iterations to the smoothed edge mask, to marginally increase width of the selected area.
- `thresh` : Threshold grayscale value for creating a binary mask from the image (how much to finally select).

Click on the **Run Interact** button to process the images.

Checking '_Save_' will also save each image to the location `./processed/{filename}` where `.` is the current directory. If such an image already exists, it will fail unless '_Overwrite_' is also checked.

In [7]:
imgStack = baseImg[SLICE[1]:SLICE[2], SLICE[0]:SLICE[3]]
imgView = None

disp.display(disp.Markdown("### Selective Blending"))

@widgets.interact_manual(edge_tol=(1,14), blur_size=(3,13,2), dil_iter=(1,5), thresh=(10,250,10), save=False, overwrite=False)

def blend(edge_tol=7, blur_size=3, dil_iter=1, thresh=120, save=False, overwrite=False):

    global imgStack, imgView

    for ipath in SOURCE_PATH:
        img = cv2.imread(ipath)[SLICE[1]:SLICE[2], SLICE[0]:SLICE[3]]

        i2outl = cv2.filter2D(img, -1, outline_kernel * edge_tol/7)
        i2olgr = cv2.cvtColor(i2outl, cv2.COLOR_BGR2GRAY)
        i2noisrm = cv2.medianBlur(i2olgr, blur_size)
        i2med1 = cv2.dilate(i2noisrm, dil_kernel, iterations=dil_iter)
        i2mask = cv2.threshold(i2med1, thresh, 255, cv2.THRESH_BINARY)[1]
        masked2 = cv2.bitwise_and(np.stack([i2mask]*3, axis=2), img)
        imgStack = cv2.max(masked2, imgStack)
        
        if save:
            if not os.path.isdir('./processed'):
                os.mkdir('./processed')
            p = './processed/'+os.path.split(ipath)[1]
            if os.path.isfile(p):
                if overwrite : os.remove(p)
                else : raise FileExistsError(f"{os.path.abspath(p)} cannot be replaced. You must enable the 'overwrite' option")
            cv2.imwrite(p, imgStack)
            print("saving", os.getcwd()+'/processed/'+os.path.split(ipath)[1])

    imgView = displayimg(imgStack, close=imgView)



### Selective Blending

interactive(children=(IntSlider(value=7, description='edge_tol', max=14, min=1), IntSlider(value=3, descriptio…

-----
> The following cell also blends the images, but without computing any mask (directly lighten the whole image, hence there are no parameters).
> This can be used as a 'control' to compare with the previous result.

Images are saved at `./processed/d_{filename}` if enabled.

In [8]:
imgStackDt = baseImg[SLICE[1]:SLICE[2], SLICE[0]:SLICE[3]]
imgViewDt  = None

disp.display(disp.Markdown("### Direct Blending"))

@widgets.interact_manual(save=False, overwrite=False)

def blend_direct(save=False, overwrite=False):

    global imgStackDt, imgViewDt

    for ipath in SOURCE_PATH:
        img = cv2.imread(ipath)[SLICE[1]:SLICE[2], SLICE[0]:SLICE[3]]
        imgStackDt = cv2.max(img, imgStackDt)
        
        if save:
            if not os.path.isdir('./processed'):
                os.mkdir('./processed')
            p = './processed/d_'+os.path.split(ipath)[1]
            if os.path.isfile(p):
                if overwrite : os.remove(p)
                else : raise FileExistsError(f"{os.path.abspath(p)} cannot be replaced. You must enable the 'overwrite' option")
            cv2.imwrite(p, imgStackDt)
            print("saving", os.getcwd()+'/processed/'+os.path.split(ipath)[1])

    imgView = displayimg(imgStackDt, close=imgViewDt)

### Direct Blending

interactive(children=(Checkbox(value=False, description='save'), Checkbox(value=False, description='overwrite'…